In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults, ARMA
from statsmodels.tsa.arima_process import ArmaProcess
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import re

In [ ]:
from functions.timeseries_functions import index_to_datetime, weekly_resample, plot_all_df_columns, plot_series,\
plot_series_save_fig, plot_series_and_differences, run_augmented_Dickey_Fuller_test, \
plot_autocorrelation, plot_partial_autocorrelation, plot_decomposition

In [ ]:
from functions.AR_MA_functions import get_AR_model, plot_AR_model, get_AR_model_order_BIC,\
plot_BIC_AR_model, get_MA_model, plot_MA_model, get_MA_train_test_predictions,\
get_MA_train_test_MSE

In [ ]:
from functions.ARIMA_functions import get_ARIMA_model, plot_ARIMA_model, plot_ARIMA_resids,\
get_ARIMA_forecast, plot_ARIMA_forecast_and_CI, plot_data_plus_ARIMA_predictions, \
test_rolling_ARIMA_forecast,get_predictions_df_and_plot_rolling_ARIMA_forecast

### import data

In [ ]:
# appointments = pd.read_csv('Appointments.csv')
appointments = pd.read_csv('./data/AppointmentsSince2015.csv')

In [ ]:
appointments.shape

In [ ]:
# break down specialty category by provider ID number
dr_ID = [7.0, 10.0, 16.0]
RNPA_ID = [3.0, 9.0, 12.0, 13.0, 14.0, 15.0, 19.0, 25.0, 27.0, 30.0]
ther_ID = [11.0, 17.0, 18.0, 23.0, 24.0, 26.0, 28.0, 29.0]

In [ ]:
appointments['Specialty'].loc[appointments['Provider'].isin(dr_ID)]= 'doctor'
appointments['Specialty'].loc[appointments['Provider'].isin(RNPA_ID)] = 'RN/PA'
appointments['Specialty'].loc[appointments['Provider'].isin(ther_ID)] = 'therapist'

In [ ]:
pd.value_counts(appointments['Specialty'])

#### remove outliers for visit Duration --> QME testing (workers comp) & initial child appointments

In [ ]:
# # convert date columns to datetime 
appointments['AppointmentCreated'] = pd.to_datetime(appointments['AppointmentCreated'], errors='coerce')
appointments['AppointmentDate'] = pd.to_datetime(appointments['AppointmentDate'], errors='coerce')

In [ ]:
appointments = appointments.set_index('AppointmentDate')

In [ ]:
appointments.to_csv('./data/appointments_full.csv')

In [ ]:
appointments.shape

In [ ]:
hours = appointments.copy()

In [ ]:
# drop rows with missing specialty
hours.dropna(subset=['Specialty'], how='all', inplace=True)

In [ ]:
hours.shape

### Use the number of hours per day per provider for each specialty

In [ ]:
hours['Hours_Spent'] = hours['AppointmentDuration'] /60

In [ ]:
hours['AppointmentDate'] = pd.to_datetime(hours.index,format='%Y-%m-%d')

In [ ]:
hours.shape
#hours.columns, hours.index

In [ ]:
duration_df = hours[['Specialty', 'AppointmentDate', 'AppointmentDuration', 'Hours_Spent']]

In [ ]:
duration_df.columns

In [ ]:
# drop appointments that are longer than 90 minutes
duration_df = duration_df[duration_df['AppointmentDuration'] <= 90]

In [ ]:
duration_df.shape

#### create training and test sets

In [ ]:
duration_train_df = duration_df['2018-03-01':]

In [ ]:
duration_test_data = duration_df['2018-05-01':'2018-02-28']

In [ ]:
duration_test_data

#### export train and test data

In [ ]:
duration_train_df.to_csv('./data/hours_training_data.csv')

In [ ]:
duration_train_df.index

In [ ]:
duration_test_data.to_csv('./data/hours_test_data.csv')

In [ ]:
duration_train_df.shape, duration_test_data.shape

In [ ]:
52881 + 4880

#### separate training data by specialty

In [ ]:
doctors = duration_train_df[duration_train_df['Specialty'] == 'doctor']
therapists = duration_train_df[duration_train_df['Specialty'] == 'therapist']
RN_PA = duration_train_df[duration_train_df['Specialty'] == 'RN/PA']

In [ ]:
doc_duration = doctors.groupby(doctors.index.date)['Hours_Spent'].sum()
RN_PA_duration = RN_PA.groupby(RN_PA.index.date)['Hours_Spent'].sum()
therapist_duration = therapists.groupby(therapists.index.date)['Hours_Spent'].sum()

In [ ]:
duration_data = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
doc_duration.shape, RN_PA_duration.shape, therapist_duration.shape

In [ ]:
doc_duration.index

In [ ]:
doc_duration.head()

In [ ]:
doc_duration.shape, RN_PA_duration.shape, therapist_duration.shape

In [ ]:
doc_duration.head()

In [ ]:
RN_PA_duration.head()

In [ ]:
therapist_duration.head()

In [ ]:
# convert index back to datetime
for item in duration_data:
    index_to_datetime(item)

In [ ]:
# resample data to weekly
doc_duration = doc_duration.resample('W-MON').sum()
RN_PA_duration = RN_PA_duration.resample('W-MON').sum()
therapist_duration = therapist_duration.resample('W-MON').sum()

In [ ]:
# drop first entry for RN/PA and therapists b/c incomplete week
# drop last entry for all categories so data ends in February
doc_duration = doc_duration[0:-1]
RN_PA_duration = RN_PA_duration[1:-1]
therapist_duration = therapist_duration[1:-1]

In [ ]:
doc_duration.head()

In [ ]:
RN_PA_duration.head()

In [ ]:
therapist_duration.head()

In [ ]:
doc_duration.shape, RN_PA_duration.shape, therapist_duration.shape

In [ ]:
training_data = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
RN_PA_duration.tail()

In [ ]:
# separate test data by specialty
dr_test_data = duration_test_data[duration_test_data['Specialty'] == 'doctor']
RN_PA_test_data = duration_test_data[duration_test_data['Specialty'] == 'RN/PA']
therapist_test_data = duration_test_data[duration_test_data['Specialty'] == 'therapist']

In [ ]:
dr_test_data = dr_test_data.groupby(dr_test_data.index.date)['Hours_Spent'].sum()
RN_PA_test_data = RN_PA_test_data.groupby(RN_PA_test_data.index.date)['Hours_Spent'].sum()
therapist_test_data = therapist_test_data.groupby(therapist_test_data.index.date)['Hours_Spent'].sum()

In [ ]:
test_data = [dr_test_data, RN_PA_test_data, therapist_test_data]

In [ ]:
# convert index back to datetime
for item in test_data:
    index_to_datetime(item)

In [ ]:
# resample data to weekly
dr_test_data = dr_test_data.resample('W-MON').sum()
RN_PA_test_data = RN_PA_test_data.resample('W-MON').sum()
therapist_test_data = therapist_test_data.resample('W-MON').sum()

In [ ]:
plot_series(series=doc_duration, figsize=(12,6), xlabel='Date', ylabel='Appointment Hours',\
            plot_name='')

In [ ]:
params = {'figure.figsize': [8,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=doc_duration, ax=axes, num_diff=2, title='Doctors')
fig.tight_layout()

In [ ]:
# test for stationarity of doctors data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(doc_duration, num_diffs=2)

In [ ]:
params = {'figure.figsize': [8,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=RN_PA_duration, ax=axes, num_diff=2, title='RN/PA')
fig.tight_layout()

In [ ]:
# test for stationarity of RN/PA data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(RN_PA_duration, num_diffs=2)

In [ ]:
# Plot therapist hours
params = {'figure.figsize': [8,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
fig, axes = plt.subplots(3, figsize=(10, 8))
plot_series_and_differences(series=therapist_duration, ax=axes, num_diff=2, title='Therapist')
fig.tight_layout()

In [ ]:
# test for stationarity of therapist data, 1st and 2nd diff
run_augmented_Dickey_Fuller_test(therapist_duration, num_diffs=2)

In [ ]:
params = {'figure.figsize': [8, 8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(doc_duration, params=params, freq=31, title='Doctors Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(RN_PA_duration, params=params, freq=31, title='RN/PA Decomposition')

In [ ]:
params = {'figure.figsize': [8,8],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_decomposition(therapist_duration, params=params, freq=31, title='Therapist Decomposition')

### AR models

#### doctors

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(doc_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours Partial Autocorrelation')

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function of first difference, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(doc_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours First Difference Partial Autocorrelation')

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
start=0
end=180
title = 'Doctors Hours Spent by Week (AR)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plt.rcParams.update(params)
fig = plot_AR_model(data=doc_duration.diff()[1:], order=(4,0), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# use BIC to confirm best number of AR components
# plot information criteria for different orders
plot_BIC_AR_model(data=doc_duration.diff()[1:], max_order_plus_one=10)

#### RN/PAs

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(RN_PA_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly RN/PA Hours Partial Autocorrelation')

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function of first difference, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(RN_PA_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly RN/PA Hours First Difference Partial Autocorrelation')

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
start=0
end=180
title = 'RN/PA Hours Spent by Week (AR)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plt.rcParams.update(params)
fig = plot_AR_model(data=RN_PA_duration.diff()[1:], order=(4,0), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# use BIC to confirm best number of AR components
# plot information criteria for different orders
plot_BIC_AR_model(data=RN_PA_duration.diff()[1:], max_order_plus_one=8)

#### therapists

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(therapist_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly Therapists Hours Partial Autocorrelation')

In [ ]:
# determine the order of the AR(p) model w/ partial autocorrelation function of first difference, alpha=width of CI
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_partial_autocorrelation(therapist_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly Therapists Hours First Difference Partial Autocorrelation')

In [ ]:
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
start=0
end=180
title = 'Therapists Hours Spent by Week (AR)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plt.rcParams.update(params)
fig = plot_AR_model(data=therapist_duration.diff()[1:], order=(6,0), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

In [ ]:
# use BIC to confirm best number of AR components
# plot information criteria for different orders
plot_BIC_AR_model(data=therapist_duration.diff()[1:], max_order_plus_one=10)

### MA models

#### doctors

In [ ]:
test_data = [dr_test_data, RN_PA_test_data, therapist_test_data]
training_data = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
def get_MA_train_test_predictions(training_data, test_data, order, start, end):
    training_data = training_data.to_frame()
    test_data = test_data.to_frame()
    results = ARMA(training_data, order=order).fit()
    forecast = results.predict(start=start, end=end).to_frame()
    all_data = pd.concat([training_data, test_data], axis=0)
    data_plus_forecast = pd.merge(left=all_data, right=forecast, how='outer', left_index=True, right_index=True)
    data_plus_forecast.columns = ['data', 'forecast']
    return forecast, data_plus_forecast

In [ ]:
def get_MA_train_test_MSE(df, data_col, pred_col, train_end, test_start, data_name=''):
    train_error_df = df.loc[:train_end]
    test_error_df = df.loc[test_start:]
    for col in train_error_df.columns:
        train_error_df = train_error_df[train_error_df[col].notnull()]
    mse_train = mean_squared_error(train_error_df[data_col], train_error_df[pred_col])
    mse_test = mean_squared_error(test_error_df[data_col], test_error_df[pred_col])
    print('train MSE: {}'.format(mse_train))
    print('test MSE: {}'.format(mse_test))

    return mse_train, mse_test

#### Test range of MA for each category, calculate train/test MSE:

In [ ]:
# doctors
start_date = '2015-01-12' 
end_date = '2018-04-30'
end_pred = '2018-07-30'

for i in range(10):
    forecast, data_plus_forecast = get_MA_train_test_predictions(training_data=doc_duration,\
                    test_data=dr_test_data, order=(0,i), start=start_date, end=end_date)
    mse_train, mse_test = get_MA_train_test_MSE(df=data_plus_forecast, data_col='data', pred_col='forecast', train_end='2018-02-26',\
                          test_start='2018-03-05', data_name='Doctor MA{}'.format(i))
    print('train MSE{}: {}'.format(i, mse_train))
    print('test MSE{}: {}'.format(i, mse_test))

In [ ]:
# plot autocorrelation function for doctors data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(doc_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours Autocorrelation')

In [ ]:
# plot autocorrelation function for first difference of doctors data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(doc_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly Doctor Hours First Difference Autocorrelation')

In [ ]:
# Plot MA model with MA order = 2 based on autocorrelation plot of first difference
params = {'figure.figsize': [12,6],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
start=0
end=180
title = 'Doctors Hours Spent by Week (MA)'
xlabel = 'Number of Hours'
ylabel = 'Date'
plt.rcParams.update(params)
fig = plot_MA_model(data=doc_duration.diff()[1:], order=(0,2), start=start, end=end, \
              title=title, xlabel=xlabel, ylabel=ylabel)

#### RN/PA

In [ ]:
# RN/PA
start_date = '2015-01-12' 
end_date = '2018-04-30'
end_pred = '2018-07-30'

for i in range(3):
    forecast, data_plus_forecast = get_MA_train_test_predictions(training_data=RN_PA_duration,\
                    test_data=RN_PA_test_data, order=(0,i), start=start_date, end=end_date)
    mse_train, mse_test = get_MA_train_test_MSE(df=data_plus_forecast, data_col='data', pred_col='forecast', train_end='2018-02-26',\
                          test_start='2018-03-05', data_name='RN/PA MA{}'.format(i))
    print('train MSE{}: {}'.format(i, mse_train))
    print('test MSE{}: {}'.format(i, mse_test))

In [ ]:
# plot autocorrelation function for RN/PA data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(RN_PA_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly RN/PA Hours Autocorrelation')

In [ ]:
# plot autocorrelation function for first difference RN/PA data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(RN_PA_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly RN/PA Hours First Difference Autocorrelation')

In [ ]:
# Plot MA model with MA order = 2 based on autocorrelation plot of first difference
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
start=0
end=180
title = 'RN/PA Hours First Difference Spent by Week (MA)'
xlabel = 'Number of Hours'
ylabel = 'Date'
fig = plot_MA_model(data=RN_PA_duration.diff()[1:], order=(0,2), start=start, end=end,\
              title=title, xlabel=xlabel, ylabel=ylabel)

#### Therapists

In [ ]:
# therapists
start_date = '2015-01-12'
end_date = '2018-04-30'
end_pred = '2018-07-30'

for i in range(12):
    forecast, data_plus_forecast = get_MA_train_test_predictions(training_data=therapist_duration,\
                    test_data=therapist_test_data, order=(0,i), start=start_date, end=end_date)
    mse_train, mse_test = get_MA_train_test_MSE(df=data_plus_forecast, data_col='data', pred_col='forecast', train_end='2018-02-26',\
                          test_start='2018-03-05', data_name='Therapists MA{}'.format(i))
    print('train MSE{}: {}'.format(i, mse_train))
    print('test MSE{}: {}'.format(i, mse_test))

In [ ]:
# plot autocorrelation function for therapist data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(therapist_duration, params=params, lags=30, alpha=0.05, \
    title='Weekly Therapist Hours Autocorrelation')

In [ ]:
# plot autocorrelation function for first difference of therapist data
params = {'figure.figsize': [6,6],'axes.labelsize': 'Medium', 'font.size': 12.0, 'lines.linewidth': 2}
plot_autocorrelation(therapist_duration.diff()[1:], params=params, lags=30, alpha=0.05, \
    title='Weekly Therapist Hours First Difference Autocorrelation')

In [ ]:
# Plot MA model with MA order = 2 based on autocorrelation plot of first difference
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
start=0
end=180
title = 'Therapists Hours Spent by Week (MA) First Difference'
xlabel = 'Number of Hours'
ylabel = 'Date'
fig = plot_MA_model(data=therapist_duration.diff()[1:], order=(0,1), start=start, end=end,\
              title=title, xlabel=xlabel, ylabel=ylabel)

### ARIMA models 

#### doctors

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
order=(4,1,2)
data = doc_duration
plot_ARIMA_model(data=data, order=order, start=1, end=180)

#### RN/PAs

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
order=(4,1,2)
data = RN_PA_duration
plot_ARIMA_model(data=data, order=order, start=1, end=180)

#### therapists

In [ ]:
params = {'figure.figsize': [12,8],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
order=(6,1,1)
data = therapist_duration
plot_ARIMA_model(data=data, order=order, start=1, end=180)

In [ ]:
def get_ARIMA_train_test_MSE(df, data_col, pred_col, train_end, test_start, data_name=''):
    """ Get ARIMAX MSE for training and test data
    Inputs:
        df: pandas dataframe of original data and ARIMAX prediction to be split into both train and test sets
        data_col = (str) name of df column containing original data
        pred_col = (str) name of df column containing model predictions
        train_end/test_start: (str) ending date for training set and starting data for test set
        data_name: (str) for labeling output
    Outputs:
        data_plus_forecast: dataframe with original data and forecast plot_all_df_columns
        forecast: just predictions
    """
    train_error_df = df.loc[:train_end]
    test_error_df = df.loc[test_start:]
    for col in train_error_df.columns:
        train_error_df = train_error_df[train_error_df[col].notnull()]
    mse_train = mean_squared_error(train_error_df[data_col], train_error_df[pred_col])
    mse_test = mean_squared_error(test_error_df[data_col], test_error_df[pred_col])
    return mse_train, mse_test

#### Weekly Hours: training and test sets

In [ ]:
training_datasets = [doc_duration, RN_PA_duration, therapist_duration]

In [ ]:
training_datasets[0][-10:]
# test_sets

In [ ]:
test_sets = [dr_test_data, RN_PA_test_data, therapist_test_data]

In [ ]:
test_sets[2][-10:]

In [ ]:
params = {'figure.figsize': [6,4],'axes.grid.axis': 'both', 'axes.grid': True, 'axes.labelsize': 'Medium', 'font.size': 12.0, \
'lines.linewidth': 2}
plt.rcParams.update(params)
i=0
providers = ['Doctors', 'RN/PAs', 'Therapists']
for data in training_datasets:
    fig = plot_series(data, plot_name=providers[i], xlabel='Date', ylabel='Hours')
    i += 1
    plt.show()